DistilBERT & ANCE

In [1]:
!pip install -U beir
!pip install -U pandas

In [2]:
import pandas as pd
import pathlib, os
from beir import util, LoggingHandler
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.datasets.data_loader import GenericDataLoader

dataset = 'scifact'
url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
out_dir = os.path.join(os.getcwd(), 'datasets')
data_path = util.download_and_unzip(url, out_dir)
corpus, queries, qrels = GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/5183 [00:00<?, ?it/s]

In [3]:
def eval_metrics(model_name, ndcg, _map, recall, precision):
  c_map, c_ndcg, c_pre, c_rec = 'MAP@10', 'NDCG@10', 'P@10', 'Recall@10'
  eval_dict = {c_map: [_map[c_map]], c_ndcg: [ndcg[c_ndcg]], c_pre: [precision[c_pre]], c_rec: [recall[c_rec]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

DistilBERT

In [4]:
model_distilbert = DRES(models.SentenceBERT('msmarco-distilbert-base-v3'), batch_size=128)
retriever_distilbert = EvaluateRetrieval(model_distilbert, score_function='cos_sim')
results_distilbert = retriever_distilbert.retrieve(corpus, queries)
ndcg_d, _map_d, recall_d, precision_d = retriever_distilbert.evaluate(
    qrels, results_distilbert, retriever_distilbert.k_values)
eval_df_distilbert = eval_metrics('DistilBERT', ndcg_d, _map_d, recall_d, precision_d)
print(eval_df_distilbert)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

             MAP@10  NDCG@10     P@10  Recall@10
DistilBERT  0.48894  0.53789  0.07567    0.67233


ANCE

In [5]:
model_ance = DRES(models.SentenceBERT('msmarco-roberta-base-ance-firstp'))
retriever_ance = EvaluateRetrieval(model_ance, score_function='dot')
results_ance = retriever_ance.retrieve(corpus, queries)
ndcg_a, _map_a, recall_a, precision_a = retriever_ance.evaluate(
    qrels, results_ance, retriever_ance.k_values)
eval_df_ance = eval_metrics('ANCE', ndcg_a, _map_a, recall_a, precision_a)
print(eval_df_ance)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

       MAP@10  NDCG@10   P@10  Recall@10
ANCE  0.46612   0.5114  0.072    0.63333


end of fun.